<h1>Chapter 09. Up and Running with TensorFlow</h1>

**TensorFlow** is an open-source machine learning framework developed by Google Brain for building and training various types of machine learning models, including deep learning neural networks. It provides a comprehensive ecosystem of tools, libraries, and resources for tasks such as data preprocessing, model building, training, deployment, and inference. TensorFlow supports both CPU and GPU computations, allowing for efficient processing of large-scale datasets and complex models. It's widely used in research, industry, and academia for a diverse range of machine learning and artificial intelligence applications.

<h2>Creating and Running a first Graph</h2>

Creating a graph

In [1]:
# Import TensorFlow version 1 compatibility module as tf
import tensorflow.compat.v1 as tf


# Disable eager execution
tf.disable_eager_execution()

# Define TensorFlow variables with initial values
x = tf.Variable(initial_value=3, name='x')
y = tf.Variable(initial_value=4, name='y')

# Define computational graph
f = x * x * y + y + 2

To run the graph, a TensorFlow session must be opened and used to initialize variables and evaluate `f`. The following code creates a session, initializes variables, computes `f`, and closes the session (freeing resources).

In [2]:
sess = tf.Session()  # create session

sess.run(x.initializer)  # initialize x
sess.run(y.initializer)  # initialize y

result = sess.run(f)  # execute the computation f
result

2024-03-21 12:47:09.792380: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


42

In [3]:
sess.close()

Rather than individually executing the initializer for each variable, it is more efficiant to utilize `global_variables_initializer()`. Repeatedly calling `sess.run` for initialization can become cumbersome; however, there exists a more streamlined approach: 

In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

result

42

`InteractiveSession()` in TensorFlow allows you to work interactively, seamlessly combining graph and execution without needing a `with` block. It's convenient for interactive environments like Jupyter notebooks, but remember to close the session manually to free up resources. 

In [5]:
sess = tf.InteractiveSession()

init.run()
result = f.eval()
result

42

In [6]:
sess.close()

<h2>Managing Graphs</h2>

In [7]:
tf.reset_default_graph()

x1 = tf.Variable(1)

x1.graph is tf.get_default_graph()

True

In [8]:
graph = tf.Graph()

with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [9]:
x2.graph is tf.get_default_graph()

False

<h2>Lifecycle of a Node Value</h2>

In [10]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [11]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15
